In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
import os
print(os.getcwd())
os.chdir("/content/gdrive/My Drive/Colab Notebooks/NewVersion")
print(os.getcwd())

Mounted at /content/gdrive
/content
/content/gdrive/My Drive/Colab Notebooks/NewVersion


In [2]:
import torch.nn.functional as F

In [3]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import numpy as np
torch.manual_seed(1)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import copy

##########################################################

label_to_ix=np.load('label_to_ix_new.npy', allow_pickle = True).item()
ix_to_label=np.load('ix_to_label_new.npy', allow_pickle = True)
training_data=np.load('training_data_new.npy', allow_pickle = True)
test_data=np.load('test_data_new.npy', allow_pickle = True)
val_data=np.load('val_data_new.npy', allow_pickle = True)
word_to_ix=np.load('word_to_ix_new.npy', allow_pickle = True).item()
ix_to_word=np.load('ix_to_word_new.npy', allow_pickle = True)
newwikivec=np.load('newwikivec_new.npy', allow_pickle = True)
wikivoc=np.load('wikivoc_new.npy', allow_pickle = True).item()



wikisize=newwikivec.shape[0]
rvocsize=newwikivec.shape[1]
wikivec=autograd.Variable(torch.FloatTensor(newwikivec))

batchsize=32


def preprocessing(data):

    new_data=[]
    for i, note, j in data:
        templabel=[0.0]*len(label_to_ix)
        for jj in j:
            if jj in wikivoc:
                templabel[label_to_ix[jj]]=1.0
        templabel=np.array(templabel,dtype=float)
        new_data.append((i, note, templabel))
    new_data=np.array(new_data)
    
    lenlist=[]
    for i in new_data:
        lenlist.append(len(i[0]))
    sortlen=sorted(range(len(lenlist)), key=lambda k: lenlist[k])  
    new_data=new_data[sortlen]
    
    batch_data=[]
    
    for start_ix in range(0, len(new_data)-batchsize+1, batchsize):
        thisblock=new_data[start_ix:start_ix+batchsize]
        mybsize= len(thisblock)
        numword=np.max([len(ii[0]) for ii in thisblock])
        main_matrix = np.zeros((mybsize, numword), dtype= np.int)
        for i in range(main_matrix.shape[0]):
            for j in range(main_matrix.shape[1]):
                try:
                    if thisblock[i][0][j] in word_to_ix:
                        main_matrix[i,j] = word_to_ix[thisblock[i][0][j]]
                    
                except IndexError:
                    pass       # because initialze with 0, so you pad with 0
    
        xxx2=[]
        yyy=[]
        for ii in thisblock:
            xxx2.append(ii[1])
            yyy.append(ii[2])
        
        xxx2=np.array(xxx2)
        yyy=np.array(yyy)
        batch_data.append((autograd.Variable(torch.from_numpy(main_matrix)),autograd.Variable(torch.FloatTensor(xxx2)),autograd.Variable(torch.FloatTensor(yyy))))
    return batch_data
batchtraining_data=preprocessing(training_data)
batchtest_data=preprocessing(test_data)
batchval_data=preprocessing(val_data)

<ipython-input-3-217e38864994>:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_data=np.array(new_data)
<ipython-input-3-217e38864994>:56: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  main_matrix = np.zeros((mybsize, numword), dtype= np.int)


In [4]:

######################################################################
# Create the model:

Embeddingsize=100
hidden_dim=200
class CNN(nn.Module):

    def __init__(self, batch_size, vocab_size, tagset_size):
        super(CNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size+1, Embeddingsize, padding_idx=0)
        self.embed_drop = nn.Dropout(p=0.2)
        
        self.hidden2tag = nn.Linear(300, tagset_size)
        
        
        self.convs1 = nn.Conv1d(Embeddingsize,100,3)
        self.convs2 = nn.Conv1d(Embeddingsize,100,4)
        self.convs3 = nn.Conv1d(Embeddingsize,100,5)
        
        
        self.layer2 = nn.Linear(Embeddingsize, 1,bias=False)
        self.embedding=nn.Linear(rvocsize,Embeddingsize)
        self.vattention=nn.Linear(Embeddingsize,Embeddingsize)
        
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, vec1, nvec, wiki, simlearning):
       
        thisembeddings=self.word_embeddings(vec1)
        thisembeddings = self.embed_drop(thisembeddings)
        thisembeddings=thisembeddings.transpose(1,2)
        
        output1=self.tanh(self.convs1(thisembeddings))
        output1=nn.MaxPool1d(output1.size()[2])(output1)
        
        output2=self.tanh(self.convs2(thisembeddings))
        output2=nn.MaxPool1d(output2.size()[2])(output2)
        
        output3=self.tanh(self.convs3(thisembeddings))
        output3=nn.MaxPool1d(output3.size()[2])(output3)
        
        output4 = torch.cat([output1,output2,output3], 1).squeeze(2)
        
        if simlearning==1:
            nvec=nvec.view(batchsize,1,-1)
            nvec=nvec.expand(batchsize,wiki.size()[0],-1)
            wiki=wiki.view(1,wiki.size()[0],-1)
            wiki=wiki.expand(nvec.size()[0],wiki.size()[1],-1)
            new=wiki*nvec
            new=self.embedding(new)
            vattention=self.sigmoid(self.vattention(new))
            new=new*vattention
            vec3=self.layer2(new)
            vec3=vec3.view(batchsize,-1)
        
       
        vec2 = self.hidden2tag(output4)
        if simlearning==1:
            tag_scores = self.sigmoid(vec2.detach()+vec3)
        else:
            tag_scores = self.sigmoid(vec2)
        
        
        return tag_scores

In [5]:
######################################################################
# Create the model:

Embeddingsize=100
hidden_dim=200
class CAML(nn.Module):

    def __init__(self, batch_size, vocab_size, tagset_size):
        super(CAML, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size+1, Embeddingsize, padding_idx=0)
        self.embed_drop = nn.Dropout(p=0.2)   
        
        
        self.convs1 = nn.Conv1d(Embeddingsize,300,10,padding=5)
        self.H=nn.Linear(300, tagset_size )   
        self.final = nn.Linear(300, tagset_size)
        
        self.layer2 = nn.Linear(Embeddingsize, 1)
        self.embedding=nn.Linear(rvocsize,Embeddingsize,bias=False)
        self.vattention=nn.Linear(Embeddingsize,Embeddingsize)
        
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
    
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, vec1, nvec, wiki, simlearning):
        
       
        thisembeddings=self.word_embeddings(vec1)
        thisembeddings = self.embed_drop(thisembeddings)
        thisembeddings=thisembeddings.transpose(1,2)
        
        
        thisembeddings=self.tanh(self.convs1(thisembeddings).transpose(1,2))  
        
        alpha=self.H.weight.matmul(thisembeddings.transpose(1,2))
        alpha=F.softmax(alpha, dim=2)
        
        m=alpha.matmul(thisembeddings)
       
        myfinal=self.final.weight.mul(m).sum(dim=2).add(self.final.bias)
        
        if simlearning==1:
            nvec=nvec.view(batchsize,1,-1)
            nvec=nvec.expand(batchsize,wiki.size()[0],-1)
            wiki=wiki.view(1,wiki.size()[0],-1)
            wiki=wiki.expand(nvec.size()[0],wiki.size()[1],-1)
            new=wiki*nvec
            new=self.embedding(new)
            vattention=self.sigmoid(self.vattention(new))
            new=new*vattention
            vec3=self.layer2(new)
            vec3=vec3.view(batchsize,-1)
        
       
        if simlearning==1:
            tag_scores = self.sigmoid(myfinal.detach()+vec3)
        else:
            tag_scores = self.sigmoid(myfinal)
        
        
        return tag_scores



topk=10

In [6]:
######################################################################
# Create the model:

Embeddingsize=100
hidden_dim=200
class LSTM(nn.Module):

    def __init__(self, batch_size, vocab_size, tagset_size):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size+1, Embeddingsize, padding_idx=0)
        self.lstm = nn.LSTM(Embeddingsize, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
        
        self.layer2 = nn.Linear(Embeddingsize, 1,bias=False)
        self.embedding=nn.Linear(rvocsize,Embeddingsize)
        self.vattention=nn.Linear(Embeddingsize,Embeddingsize,bias=False)
        
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.embed_drop = nn.Dropout(p=0.2)
    
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, batchsize, self.hidden_dim).cuda()),
                autograd.Variable(torch.zeros(1, batchsize, self.hidden_dim)).cuda())

    
    def forward(self, vec1, nvec, wiki, simlearning):
      
        thisembeddings=self.word_embeddings(vec1).transpose(0,1)
        thisembeddings = self.embed_drop(thisembeddings)
       
        if simlearning==1:
            nvec=nvec.view(batchsize,1,-1)
            nvec=nvec.expand(batchsize,wiki.size()[0],-1)
            wiki=wiki.view(1,wiki.size()[0],-1)
            wiki=wiki.expand(nvec.size()[0],wiki.size()[1],-1)
            new=wiki*nvec
            new=self.embedding(new)
            vattention=self.sigmoid(self.vattention(new))
            new=new*vattention
            vec3=self.layer2(new)
            vec3=vec3.view(batchsize,-1)
        
        
        
        lstm_out, self.hidden = self.lstm(
            thisembeddings, self.hidden)
        
        lstm_out=lstm_out.transpose(0,2).transpose(0,1)
        
        output1=nn.MaxPool1d(lstm_out.size()[2])(lstm_out).view(batchsize,-1)
        
        vec2 = self.hidden2tag(output1)
        if simlearning==1:
            tag_scores = self.sigmoid(vec2.detach()+vec3)
        else:
            tag_scores = self.sigmoid(vec2)
        
        
        return tag_scores

In [7]:

######################################################################
# Create the model:

Embeddingsize=100
hidden_dim=200
class LSTMattn(nn.Module):

    def __init__(self, batch_size, vocab_size, tagset_size):
        super(LSTMattn, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size+1, Embeddingsize, padding_idx=0)
        self.lstm = nn.LSTM(Embeddingsize, hidden_dim)
        self.hidden = self.init_hidden()
        
        self.H=nn.Linear(hidden_dim, tagset_size )  
        self.final = nn.Linear(hidden_dim, tagset_size)
        
        self.layer2 = nn.Linear(Embeddingsize, 1,bias=False)
        self.embedding=nn.Linear(rvocsize,Embeddingsize)
        self.vattention=nn.Linear(Embeddingsize,Embeddingsize,bias=False)
        
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.embed_drop = nn.Dropout(p=0.2)
    
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, batchsize, self.hidden_dim).cuda()),
                autograd.Variable(torch.zeros(1, batchsize, self.hidden_dim)).cuda())

    
    def forward(self, vec1, nvec, wiki, simlearning):
        
        
        thisembeddings=self.word_embeddings(vec1).transpose(0,1)
        thisembeddings = self.embed_drop(thisembeddings)
        
        
        if simlearning==1:
            nvec=nvec.view(batchsize,1,-1)
            nvec=nvec.expand(batchsize,wiki.size()[0],-1)
            wiki=wiki.view(1,wiki.size()[0],-1)
            wiki=wiki.expand(nvec.size()[0],wiki.size()[1],-1)
            new=wiki*nvec
            new=self.embedding(new)
            vattention=self.sigmoid(self.vattention(new))
            new=new*vattention
            vec3=self.layer2(new)
            vec3=vec3.view(batchsize,-1)
        
        
        lstm_out, self.hidden = self.lstm(
            thisembeddings, self.hidden)
        
        
        
        lstm_out=lstm_out.transpose(0,1)

        alpha=self.H.weight.matmul(lstm_out.transpose(1,2))
        alpha=F.softmax(alpha, dim=2)
        
        m=alpha.matmul(lstm_out)
        
        myfinal=self.final.weight.mul(m).sum(dim=2).add(self.final.bias)
        
        
        if simlearning==1:
            tag_scores = self.sigmoid(myfinal.detach()+vec3)
        else:
            tag_scores = self.sigmoid(myfinal)
        
        
        return tag_scores

In [8]:
CAML_model = CAML(batchsize, len(word_to_ix), len(label_to_ix))
CAML_model.load_state_dict(torch.load("CAML_model_new"))
CAML_model

CAML(
  (word_embeddings): Embedding(47962, 100, padding_idx=0)
  (embed_drop): Dropout(p=0.2, inplace=False)
  (convs1): Conv1d(100, 300, kernel_size=(10,), stride=(1,), padding=(5,))
  (H): Linear(in_features=300, out_features=344, bias=True)
  (final): Linear(in_features=300, out_features=344, bias=True)
  (layer2): Linear(in_features=100, out_features=1, bias=True)
  (embedding): Linear(in_features=12881, out_features=100, bias=False)
  (vattention): Linear(in_features=100, out_features=100, bias=True)
  (sigmoid): Sigmoid()
  (tanh): Tanh()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [9]:
CNN_model = CNN(batchsize, len(word_to_ix), len(label_to_ix))
CNN_model.load_state_dict(torch.load("CNN_model_new"))
CNN_model

CNN(
  (word_embeddings): Embedding(47962, 100, padding_idx=0)
  (embed_drop): Dropout(p=0.2, inplace=False)
  (hidden2tag): Linear(in_features=300, out_features=344, bias=True)
  (convs1): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
  (convs2): Conv1d(100, 100, kernel_size=(4,), stride=(1,))
  (convs3): Conv1d(100, 100, kernel_size=(5,), stride=(1,))
  (layer2): Linear(in_features=100, out_features=1, bias=False)
  (embedding): Linear(in_features=12881, out_features=100, bias=True)
  (vattention): Linear(in_features=100, out_features=100, bias=True)
  (sigmoid): Sigmoid()
  (tanh): Tanh()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [10]:
LSTM_model = LSTM(batchsize, len(word_to_ix), len(label_to_ix))
LSTM_model.load_state_dict(torch.load("LSTM_model_new"))
LSTM_model

LSTM(
  (word_embeddings): Embedding(47962, 100, padding_idx=0)
  (lstm): LSTM(100, 200)
  (hidden2tag): Linear(in_features=200, out_features=344, bias=True)
  (layer2): Linear(in_features=100, out_features=1, bias=False)
  (embedding): Linear(in_features=12881, out_features=100, bias=True)
  (vattention): Linear(in_features=100, out_features=100, bias=False)
  (softmax): Softmax(dim=None)
  (sigmoid): Sigmoid()
  (tanh): Tanh()
  (embed_drop): Dropout(p=0.2, inplace=False)
)

In [11]:
LSTMattn_model = LSTMattn(batchsize, len(word_to_ix), len(label_to_ix))
LSTMattn_model.load_state_dict(torch.load("LSTMattn_model_new"))
LSTMattn_model

LSTMattn(
  (word_embeddings): Embedding(47962, 100, padding_idx=0)
  (lstm): LSTM(100, 200)
  (H): Linear(in_features=200, out_features=344, bias=True)
  (final): Linear(in_features=200, out_features=344, bias=True)
  (layer2): Linear(in_features=100, out_features=1, bias=False)
  (embedding): Linear(in_features=12881, out_features=100, bias=True)
  (vattention): Linear(in_features=100, out_features=100, bias=False)
  (softmax): Softmax(dim=None)
  (sigmoid): Sigmoid()
  (embed_drop): Dropout(p=0.2, inplace=False)
)

In [12]:
KSI_CAML_model = CAML(batchsize, len(word_to_ix), len(label_to_ix))
KSI_CAML_model.load_state_dict(torch.load("KSI_CAML_model_new"))
KSI_CNN_model = CNN(batchsize, len(word_to_ix), len(label_to_ix))
KSI_CNN_model.load_state_dict(torch.load("KSI_CNN_model_new"))
KSI_LSTM_model = LSTM(batchsize, len(word_to_ix), len(label_to_ix))
KSI_LSTM_model.load_state_dict(torch.load("KSI_LSTM_model_new"))
KSI_LSTMattn_model = LSTMattn(batchsize, len(word_to_ix), len(label_to_ix))
KSI_LSTMattn_model.load_state_dict(torch.load("KSI_LSTMattn_model_new"))

<All keys matched successfully>

In [13]:
targets = [batch[2] for batch in batchtest_data]
targets_train = [batch[2] for batch in batchtraining_data]
targets_val = [batch[2] for batch in batchval_data]
targets.extend(targets_train)
targets.extend(targets_val)

In [14]:
label_freq = torch.stack(targets).sum(dim = 0).sum(dim = 0)

In [15]:
label_divide = [ [], [], [], [], []]

Label divide index 0 is the labels that occur in the data 1-10 times.

Label divide index 1 is the labels that occur in the data 11-50 times.

Label divide index 2 is the labels that occur in the data 51-100 times.

Label divide index 3 is the labels that occur in the data 100-500 times.

Label divide index 4 is the labels that occur in the data 501+ times.

In [16]:
for count, label in enumerate(label_freq):
  if label > 500:
    label_divide[4].append(count)
  elif label > 100:
    label_divide[3].append(count)
  elif label > 50:
    label_divide[2].append(count)
  elif label > 10:
    label_divide[1].append(count)
  else:
    label_divide[0].append(count)

In [17]:
[len(labels) for labels in label_divide]

[80, 73, 25, 82, 84]

In [18]:
def testmodel(model, sim):
    model.cuda()
    loss_function = nn.BCELoss()
    model.eval()
    recall=[]
    lossestest = []
    
    y_true=[]
    y_scores=[]
    
    for inputs in batchtest_data:
        with torch.no_grad():
          targets = inputs[2].cuda()
          
          tag_scores = model(inputs[0].cuda(),inputs[1].cuda() ,wikivec.cuda(),sim)

          loss = loss_function(tag_scores, targets)
          
          targets=targets.data.cpu().numpy()
          tag_scores= tag_scores.data.cpu().numpy()
          
          
          lossestest.append(loss.data.cpu().numpy().mean())
          y_true.append(targets)
          y_scores.append(tag_scores)
        
        for iii in range(0,len(tag_scores)):
            temp={}
            for iiii in range(0,len(tag_scores[iii])):
                temp[iiii]=tag_scores[iii][iiii]
            temp1=[(k, temp[k]) for k in sorted(temp, key=temp.get, reverse=True)]
            thistop=int(np.sum(targets[iii]))
            hit=0.0
            
            for ii in temp1[0:max(thistop,topk)]:
                if targets[iii][ii[0]]==1.0:
                    hit=hit+1
            if thistop!=0:
                recall.append(hit/thistop)
    y_true=np.concatenate(y_true,axis=0)
    y_scores=np.concatenate(y_scores,axis=0)
    y_true=y_true.T
    y_scores=y_scores.T
    macro_AUC = []
    for labels in label_divide:
      y_true_temp = y_true[labels]
      y_scores_temp = y_scores[labels]
      temptrue=[]
      tempscores=[]
      for col in range(0,len(y_true_temp)):
          if np.sum(y_true_temp[col])!=0:
              temptrue.append(y_true_temp[col])
              tempscores.append(y_scores_temp[col])
      temptrue=np.array(temptrue)
      tempscores=np.array(tempscores)
      y_true_temp=temptrue.T
      y_scores_temp=tempscores.T
      macro_AUC.append(roc_auc_score(y_true_temp, y_scores_temp,average='macro'))
    return macro_AUC

In [19]:
RNN_model_results = testmodel(LSTM_model, 0)
KSI_RNN_model_results = testmodel(KSI_LSTM_model, 1)
RNNattn_model_results = testmodel(LSTMattn_model, 0)
KSI_RNNattn_model_results = testmodel(KSI_LSTMattn_model, 1)
CNN_model_results = testmodel(CNN_model, 0)
KSI_CNN_model_results = testmodel(KSI_CNN_model, 1)
CAML_model_results = testmodel(CAML_model, 0)
KSI_CAML_model_reults = testmodel(KSI_CAML_model, 1)

In [20]:
results = np.array([RNN_model_results,
KSI_RNN_model_results,
RNNattn_model_results,
KSI_RNNattn_model_results, 
CNN_model_results,
KSI_CNN_model_results,
CAML_model_results,
KSI_CAML_model_reults])

In [21]:
import pandas as pd

In [22]:
result = pd.DataFrame(results, index=["RNN", "KSI+RNN", "RNNatt", "KSI+RNNatt", "CNN", "KSI+CNN", "CAML", "KSI+CAML"],
                             columns=['1-10', '11-50','51-100','101-500','>500'])

In [23]:
print(result.round(4).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &    1-10 &   11-50 &  51-100 &  101-500 &    >500 \\
\midrule
RNN        &  0.7407 &  0.7565 &  0.8089 &   0.8444 &  0.9048 \\
KSI+RNN    &  0.7945 &  0.8243 &  0.8846 &   0.8986 &  0.9219 \\
RNNatt     &  0.6771 &  0.7894 &  0.8528 &   0.8928 &  0.9232 \\
KSI+RNNatt &  0.7765 &  0.8459 &  0.8997 &   0.9231 &  0.9318 \\
CNN        &  0.7277 &  0.7800 &  0.8158 &   0.8606 &  0.9090 \\
KSI+CNN    &  0.7775 &  0.8297 &  0.8880 &   0.8981 &  0.9234 \\
CAML       &  0.7267 &  0.7764 &  0.8744 &   0.9113 &  0.9337 \\
KSI+CAML   &  0.7960 &  0.8507 &  0.9117 &   0.9274 &  0.9360 \\
\bottomrule
\end{tabular}



<ipython-input-23-cb30f152a4d1>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(result.round(4).to_latex())
